# 01 — Pandas  → *Spatial-Ready*

Obiettivi (lentamente, per passi piccolissimi):
- Capire cos'è un **DataFrame** e come si esplora.
- Imparare **selezioni**, **filtri**, **nuove colonne**, **missing**, **ordinamenti**.
- Fare un **groupby** semplice e un **merge** (join) con una tabella di lookup.
- Preparare file **puliti** (CSV/Parquet) usando **percorsi relativi**.
- Applicare gli stessi passi su un mini caso reale: **GTFS** (fermate/linee).


## 0) Percorsi relativi (niente path con username)
Il notebook rileva automaticamente la cartella `Corso_PyQGIS` e usa `Corso_PyQGIS/data/...`.

In [3]:
from pathlib import Path

# Cartella del notebook (senza risalire di livello)
NB_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

# Radice dati DENTRO la cartella del notebook
DATA_ROOT = NB_DIR / "data"

# Se vuoi tenere la stessa struttura "lezione"
LEZIONE = "data_lezione_01"
DATA_RAW = DATA_ROOT / LEZIONE / "raw"
DATA_RAW_GTFS = DATA_RAW / "gtfs"
DATA_CLEAN = DATA_ROOT / LEZIONE / "clean"

# Crea tutto qui, localmente
for p in [DATA_ROOT, DATA_RAW, DATA_RAW_GTFS, DATA_CLEAN]:
    p.mkdir(parents=True, exist_ok=True)

print("Notebook dir:", NB_DIR)
print("Data root   :", DATA_ROOT)
print("GTFS raw    :", DATA_RAW_GTFS)  # file che scaricheremo durante la lezione
print("Clean out   :", DATA_CLEAN)


Notebook dir: /home/ciro/Downloads/Corso_PyQGIS
Data root   : /home/ciro/Downloads/Corso_PyQGIS/data
GTFS raw    : /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_01/raw/gtfs
Clean out   : /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_01/clean


## 1) Pandas **da zero** (micro-steps)
Partiamo dalle basi e **vediamo subito gli output** (con `print()` e `head()`) ad ogni passo.


In [6]:
import pandas as pd, numpy as np
print("pandas:", pd.__version__, "| numpy:", np.__version__)

pandas: 2.2.3 | numpy: 2.2.6


### 1.1 Creiamo un DataFrame semplice

In [4]:
df = pd.DataFrame({
    "city": ["A","A","B","B","C"],
    "value": [10, 12, 7, None, 5],
    "when": pd.to_datetime(["2025-10-01","2025-10-02","2025-10-01","2025-10-02","2025-10-01"])
})
print("Shape:", df.shape)
print("Columns:", list(df.columns))
print("\nHEAD:")
display(df.head())

Shape: (5, 3)
Columns: ['city', 'value', 'when']

HEAD:


,city,value,when
0,A,10.0,2025-10-01
1,A,12.0,2025-10-02
2,B,7.0,2025-10-01
3,B,NaN,2025-10-02
4,C,5.0,2025-10-01


### 1.2 Tipi di dato, `info()` e `describe()`

In [5]:
print(df.dtypes)
print("\nINFO:")
display(df.info())
print("\nDESCRIBE (solo numeriche):")
try:
    display(df.describe(numeric_only=True))
except TypeError:
    display(df.select_dtypes(include='number').describe())

city             object
value           float64
when     datetime64[ns]
dtype: object

INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   city    5 non-null      object        
 1   value   4 non-null      float64       
 2   when    5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 248.0+ bytes


None


DESCRIBE (solo numeriche):


,value
count,4.000000
mean,8.500000
std,3.109126
min,5.000000
25%,6.500000
50%,8.500000
75%,10.500000
max,12.000000


### 1.3 Selezioni base (colonne e righe)

In [6]:
print("Colonna city:")
display(df["city"].head())

print("\nDue colonne (city, value):")
display(df[["city","value"]].head())

print("\nPrime 3 righe con iloc:")
display(df.iloc[:3])

Colonna city:


0    A
1    A
2    B
3    B
4    C
Name: city, dtype: object


Due colonne (city, value):


,city,value
0,A,10.0
1,A,12.0
2,B,7.0
3,B,NaN
4,C,5.0



Prime 3 righe con iloc:


,city,value,when
0,A,10.0,2025-10-01
1,A,12.0,2025-10-02
2,B,7.0,2025-10-01


### 1.4 Filtri booleani (>, ==, &, |)

In [7]:
filtro = df["value"] >= 10
print("Filtro (value >= 10):")
display(filtro.head())

filtrato = df[filtro]
print("\nRighe che passano il filtro:")
display(filtrato)

# combinazioni logiche
combo = df[(df["value"] >= 10) & (df["city"] == "A")]
print("\nFiltro combinato (value>=10 E city=='A'):")
display(combo)

Filtro (value >= 10):


0     True
1     True
2    False
3    False
4    False
Name: value, dtype: bool


Righe che passano il filtro:


,city,value,when
0,A,10.0,2025-10-01
1,A,12.0,2025-10-02



Filtro combinato (value>=10 E city=='A'):


,city,value,when
0,A,10.0,2025-10-01
1,A,12.0,2025-10-02


### 1.5 `loc` vs `iloc` (etichette vs posizioni)

In [8]:
print("loc su etichette di riga/colonna:")
display(df.loc[df["city"]=="B", ["city","value"]])

print("\niloc su posizioni (prime 2 righe, prime 2 colonne):")
display(df.iloc[:2, :2])

loc su etichette di riga/colonna:


,city,value
2,B,7.0
3,B,NaN



iloc su posizioni (prime 2 righe, prime 2 colonne):


,city,value
0,A,10.0
1,A,12.0


### 1.6 Nuove colonne (vettoriale)

In [9]:
df2 = df.copy()
df2["day"] = df2["when"].dt.date
df2["value2"] = df2["value"].fillna(0) * 2  # niente apply riga-per-riga
print("Nuove colonne 'day' e 'value2':")
display(df2.head())

Nuove colonne 'day' e 'value2':


,city,value,when,day,value2
0,A,10.0,2025-10-01,2025-10-01,20.0
1,A,12.0,2025-10-02,2025-10-02,24.0
2,B,7.0,2025-10-01,2025-10-01,14.0
3,B,NaN,2025-10-02,2025-10-02,0.0
4,C,5.0,2025-10-01,2025-10-01,10.0


### 1.7 Missing values: `isna`, `fillna`, `dropna`

In [10]:
print("Quanti NaN in value?:", df["value"].isna().sum())
print("\nfillna(0):")
display(df["value"].fillna(0).head())

print("\ndropna su 'value':")
display(df.dropna(subset=["value"]))

Quanti NaN in value?: 1

fillna(0):


0    10.0
1    12.0
2     7.0
3     0.0
4     5.0
Name: value, dtype: float64


dropna su 'value':


,city,value,when
0,A,10.0,2025-10-01
1,A,12.0,2025-10-02
2,B,7.0,2025-10-01
4,C,5.0,2025-10-01


### 1.8 Ordinamento, `unique`, `value_counts`

In [11]:
print("Ordinamento per city asc, value desc:")
display(df.sort_values(["city","value"], ascending=[True, False]))

print("\nValori unici in city:", df["city"].unique())
print("\nFrequenze in city:")
display(df["city"].value_counts())

Ordinamento per city asc, value desc:


,city,value,when
1,A,12.0,2025-10-02
0,A,10.0,2025-10-01
2,B,7.0,2025-10-01
3,B,NaN,2025-10-02
4,C,5.0,2025-10-01



Valori unici in city: ['A' 'B' 'C']

Frequenze in city:


city
A    2
B    2
C    1
Name: count, dtype: int64

### 1.9 GroupBy (conteggi e media)

In [12]:
g = df.groupby("city")["value"].agg(count="count", mean="mean").reset_index()
print("GroupBy per city (count/mean):")
display(g)

GroupBy per city (count/mean):


,city,count,mean
0,A,2,11.0
1,B,1,7.0
2,C,1,5.0


### 1.10 Pivot 

Se invece vogliamo *rimodellare* i dati, trasformando i valori di una colonna (es. `when`) in *nuove colonne*, usiamo `pivot_table`.

In [13]:
# Riempiamo il NaN per vederlo meglio nel pivot
df_filled = df.fillna(0) 

# Indice: city, Colonne: when, Valori: value
df_pivot = df_filled.pivot_table(
    index="city", 
    columns="when", 
    values="value"
)

display(df_pivot)

when,2025-10-01,2025-10-02
city,,
A,10.0,12.0
B,7.0,0.0
C,5.0,NaN


### 1.11 Merge (join) con tabella di lookup

In [14]:
lookup = pd.DataFrame({"city":["A","B","C"], "region":["N","N","S"]})
m = df.merge(lookup, on="city", how="left")
print("Join LEFT su 'city':")
display(m.head())

Join LEFT su 'city':


,city,value,when,region
0,A,10.0,2025-10-01,N
1,A,12.0,2025-10-02,N
2,B,7.0,2025-10-01,N
3,B,NaN,2025-10-02,N
4,C,5.0,2025-10-01,S


---
## 2) Mini-caso reale: **GTFS** (fermate/linee)
Faremo gli stessi passi: lettura → esplora → pulisci → arricchisci → salva.


### 2.1 Download GTFS reale 

In [1]:
import urllib.request, zipfile, shutil

# GTFS_URL = "https://cdn.mbta.com/MBTA_GTFS.zip"
# ZIP_PATH = DATA_RAW_GTFS / "gtfs_source.zip"
# EXTRACT_DIR = DATA_RAW_GTFS / "gtfs_source"

# def try_download(url, dest):
#     try:
#         print("Scarico:", url)
#         with urllib.request.urlopen(url, timeout=30) as resp, open(dest, "wb") as f:
#             f.write(resp.read())
#         print("OK:", dest)
#         return True
#     except Exception as e:
#         print("Download fallito (scaricare lo zip manualmente):", e)
#         return False

# ok = True
# if not ZIP_PATH.exists():
#     ok = try_download(GTFS_URL, ZIP_PATH)

# found = {}
# if ok:
#     try:
#         if EXTRACT_DIR.exists():
#             shutil.rmtree(EXTRACT_DIR)
#         EXTRACT_DIR.mkdir(parents=True, exist_ok=True)
#         with zipfile.ZipFile(ZIP_PATH, "r") as z:
#             z.extractall(EXTRACT_DIR)
#         for name in ["stops.txt","routes.txt","trips.txt","stop_times.txt"]:
#             p = EXTRACT_DIR / name
#             found[name] = p if p.exists() else None
#         print("Estratto in:", EXTRACT_DIR)
#     except Exception as e:
#         print("Estrazione fallita:", e)
#         found = {}

# use_demo = not all(found.get(n) for n in ["stops.txt","routes.txt","trips.txt","stop_times.txt"])
# files = {}
# if use_demo:
#     print("→ Uso i file DEMO in data/raw/gtfs/ (inclusi nel pacchetto).")
#     files["stops"] = DATA_RAW_GTFS / "stops.txt"
#     files["routes"] = DATA_RAW_GTFS / "routes.txt"
#     files["trips"] = DATA_RAW_GTFS / "trips.txt"
#     files["stop_times"] = DATA_RAW_GTFS / "stop_times.txt"
# else:
#     files["stops"] = found["stops.txt"]
#     files["routes"] = found["routes.txt"]
#     files["trips"] = found["trips.txt"]
#     files["stop_times"] = found["stop_times.txt"]

# print("File scelti:")
# for k, p in files.items():
#     print(f" - {k:>10}: {p}")

In [4]:
from pathlib import Path
import shutil, zipfile


GTFS_URL = "https://cdn.mbta.com/MBTA_GTFS.zip"
ZIP_PATH = DATA_RAW_GTFS / "gtfs_source.zip"
EXTRACT_DIR = DATA_RAW_GTFS / "gtfs_source"


# 1) assicurati che la cartella di destinazione esista
ZIP_PATH.parent.mkdir(parents=True, exist_ok=True)

# 2) cerca uno ZIP caricato manualmente nei posti tipici di Basthon
candidates = [
    ZIP_PATH,  # magari già presente
    Path("/home/pyodide/MBTA_GTFS.zip"),
    Path("/home/pyodide/gtfs.zip"),
    Path("/home/pyodide/Downloads/MBTA_GTFS.zip"),
    Path("/home/pyodide/Downloads/gtfs.zip"),
]

found_upload = next((p for p in candidates if p.exists()), None)

if not found_upload:
    raise RuntimeError(
        "Nessuno ZIP trovato. Carica 'MBTA_GTFS.zip' con Upload (finisce in /home/pyodide/) e riesegui."
    )

# 3) copia/rinomina nello ZIP atteso dal tuo codice
if found_upload != ZIP_PATH:
    shutil.copy(found_upload, ZIP_PATH)
    print("Trovato upload:", found_upload)
print("ZIP pronto:", ZIP_PATH)

# 4) estrazione pulita
if EXTRACT_DIR.exists():
    shutil.rmtree(EXTRACT_DIR)
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)
with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(EXTRACT_DIR)
print("Estratto in:", EXTRACT_DIR)

files = {
    "stops": EXTRACT_DIR / "stops.txt",
    "routes": EXTRACT_DIR / "routes.txt",
    "trips": EXTRACT_DIR / "trips.txt",
    "stop_times": EXTRACT_DIR / "stop_times.txt",
}


ZIP pronto: /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_01/raw/gtfs/gtfs_source.zip
Estratto in: /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_01/raw/gtfs/gtfs_source


### 2.2 Leggiamo **una tabella per volta** e guardiamo `head()`

In [7]:
stops = pd.read_csv(files["stops"])
print("stops shape:", stops.shape); display(stops.head())

stops shape: (10286, 19)


,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type
0,1,1.0,Washington St opp Ruggles St,NaN,NaN,NaN,42.330957,-71.082754,ExpressBus-Downtown,NaN,https://www.mbta.com/stops/1,NaN,0,NaN,1,Boston,Washington Street,Ruggles Street,3.0
1,10,10.0,Theo Glynn Way @ Newmarket Sq,NaN,NaN,NaN,42.330555,-71.068787,LocalBus,NaN,https://www.mbta.com/stops/10,NaN,0,NaN,1,Boston,Theodore Glynn Way,Newmarket Square,3.0
2,10000,10000.0,Tremont St opp Temple Pl,NaN,NaN,NaN,42.355692,-71.062911,LocalBus,NaN,https://www.mbta.com/stops/10000,NaN,0,NaN,1,Boston,Tremont Street,Temple Place,3.0
3,10003,10003.0,Albany St opp Randall St,NaN,NaN,NaN,42.331591,-71.076237,LocalBus,NaN,https://www.mbta.com/stops/10003,NaN,0,NaN,1,Boston,Albany Street,Randall Street,3.0
4,10005,10005.0,Albany St opp E Concord St,NaN,NaN,NaN,42.335017,-71.071280,LocalBus,NaN,https://www.mbta.com/stops/10005,NaN,0,NaN,1,Boston,Albany Street,NaN,3.0


In [17]:
routes = pd.read_csv(files["routes"])
print("routes shape:", routes.shape); display(routes.head())

routes shape: (398, 14)


,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order,route_fare_class,line_id,listed_route,network_id
0,Red,1,NaN,Red Line,Rapid Transit,1,https://www.mbta.com/schedules/Red,DA291C,FFFFFF,10010,Rapid Transit,line-Red,NaN,rapid_transit
1,Mattapan,1,NaN,Mattapan Trolley,Rapid Transit,0,https://www.mbta.com/schedules/Mattapan,DA291C,FFFFFF,10011,Rapid Transit,line-Mattapan,NaN,m_rapid_transit
2,Orange,1,NaN,Orange Line,Rapid Transit,1,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020,Rapid Transit,line-Orange,NaN,rapid_transit
3,Green-B,1,B,Green Line B,Rapid Transit,0,https://www.mbta.com/schedules/Green-B,00843D,FFFFFF,10032,Rapid Transit,line-Green,NaN,rapid_transit
4,Green-C,1,C,Green Line C,Rapid Transit,0,https://www.mbta.com/schedules/Green-C,00843D,FFFFFF,10033,Rapid Transit,line-Green,NaN,rapid_transit


In [18]:
trips = pd.read_csv(files["trips"])
print("trips shape:", trips.shape); display(trips.head())

trips shape: (87511, 12)


,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed
0,1,BUS42025-hbc45fr1-Weekday-02,71377407,Harvard,NaN,0,C01-1,010150,1,NaN,1-_-0,1
1,1,BUS42025-hbc45fr1-Weekday-02,71377409,Harvard,NaN,0,C01-8,010150,1,NaN,1-_-0,1
2,1,BUS42025-hbc45fr1-Weekday-02,71377411,Harvard,NaN,0,C01-3,010150,1,NaN,1-_-0,1
3,1,BUS42025-hbc45fr1-Weekday-02,71377415,Harvard,NaN,0,C01-1,010150,1,NaN,1-_-0,1
4,1,BUS42025-hbc45fr1-Weekday-02,71377420,Harvard,NaN,0,C01-7,010150,1,NaN,1-_-0,1


In [19]:
stop_times = pd.read_csv(files["stop_times"])
print("stop_times shape:", stop_times.shape); display(stop_times.head())

/tmp/ipykernel_13244/175215017.py:1: DtypeWarning: Columns (0,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv(files["stop_times"])


stop_times shape: (2133080, 12)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,continuous_pickup,continuous_drop_off
0,70505896,05:15:00,05:15:00,70036,1,NaN,0,1,0,ogmnl,NaN,NaN
1,70505896,05:16:00,05:16:00,70034,10,NaN,0,0,0,mlmnl,NaN,NaN
2,70505896,05:20:00,05:20:00,70032,20,NaN,0,0,0,welln,NaN,NaN
3,70505896,05:22:00,05:22:00,70278,30,NaN,0,0,0,astao,NaN,NaN
4,70505896,05:24:00,05:24:00,70030,40,NaN,0,0,0,sull,NaN,NaN


Esplorazione: Filtri testuali (`str.contains`)

Possiamo esplorare i dati testuali, ad esempio cercando tutte le fermate che contengono una parola specifica nel nome.

In [20]:
# Cerchiamo tutte le fermate che contengono 'Street' nel nome
# na=False serve a ignorare i NaN (se ce ne fossero) invece di dare errore
stops_street = stops[stops["stop_name"].str.contains("Street", na=False)] #contains()

print(f"Trovate {len(stops_street)} fermate con 'Street' nel nome:")
display(stops_street[["stop_id", "stop_name"]].head())

Trovate 183 fermate con 'Street' nel nome:


,stop_id,stop_name
2019,293,W 7th @ B Street
3451,46495,Hyde Park Ave opp Florian Street
3461,467,Washington Street @ Four Corners / Geneva Station
3510,475,Washington Street @ Four Corners / Geneva Station
3900,52370,Green Street Station - Shuttle


### 2.3 `snake_case` e tipizzazione **visibili** passo per passo

In [21]:
def snake_case(df):
    df = df.copy()
    before = list(df.columns)
    df.columns = (df.columns.str.strip()
                            .str.replace(" ", "_", regex=False)
                            .str.replace("-", "_", regex=False)
                            .str.lower())
    after = list(df.columns)
    print("Colonne prima:", before)
    print("Colonne dopo :", after)
    return df

stops = snake_case(stops)
routes = snake_case(routes)
trips = snake_case(trips)
stop_times = snake_case(stop_times)

print("\\nDtypes PRIMA:"); print(stops.dtypes)
# tipizzazione
if "stop_id" in stops.columns: stops["stop_id"] = stops["stop_id"].astype("string")
if "stop_name" in stops.columns: stops["stop_name"] = stops["stop_name"].astype("string")
for col in ["stop_lat","stop_lon"]:
    if col in stops.columns:
        stops[col] = pd.to_numeric(stops[col], errors="coerce")
print("\\nDtypes DOPO:"); print(stops.dtypes)

Colonne prima: ['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'platform_code', 'platform_name', 'stop_lat', 'stop_lon', 'zone_id', 'stop_address', 'stop_url', 'level_id', 'location_type', 'parent_station', 'wheelchair_boarding', 'municipality', 'on_street', 'at_street', 'vehicle_type']
Colonne dopo : ['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'platform_code', 'platform_name', 'stop_lat', 'stop_lon', 'zone_id', 'stop_address', 'stop_url', 'level_id', 'location_type', 'parent_station', 'wheelchair_boarding', 'municipality', 'on_street', 'at_street', 'vehicle_type']
Colonne prima: ['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color', 'route_sort_order', 'route_fare_class', 'line_id', 'listed_route', 'network_id']
Colonne dopo : ['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color', 'route_sort_order', 'rout

### 2.4 Validiamo coordinate e rimuoviamo righe invalide

In [22]:
before = len(stops)
mask = (
    stops.get("stop_lat", pd.Series(dtype=float)).between(-90, 90) &
    stops.get("stop_lon", pd.Series(dtype=float)).between(-180, 180)
)
stops = stops[mask].dropna(subset=["stop_lat","stop_lon"]).drop_duplicates()
after = len(stops)
print(f"Righe prima: {before} -> dopo: {after} (rimosse {before-after})")

Righe prima: 10286 -> dopo: 9448 (rimosse 838)


### 2.5 Verifica **id univoci** su `stop_id`

In [23]:
dups = stops["stop_id"].duplicated().sum()
print("Duplicati su stop_id:", dups)
assert dups == 0, "stop_id non univoco" 

Duplicati su stop_id: 0


### 2.6 Join graduale: `stop_times → trips → routes`

In [24]:
print("Join 1/2: stop_times + trips (on=trip_id)")
st_trip = stop_times.merge(trips, on="trip_id", how="left")
print("Shape:", st_trip.shape); display(st_trip.head())

Join 1/2: stop_times + trips (on=trip_id)
Shape: (2133080, 23)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,...,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed
0,70505896,05:15:00,05:15:00,70036,1,NaN,0,1,0,ogmnl,...,RTL42025-hmo45011-Weekday-01,Forest Hills,NaN,0.0,O903_-1,903_0026,1.0,NaN,Orange-3-0,0.0
1,70505896,05:16:00,05:16:00,70034,10,NaN,0,0,0,mlmnl,...,RTL42025-hmo45011-Weekday-01,Forest Hills,NaN,0.0,O903_-1,903_0026,1.0,NaN,Orange-3-0,0.0
2,70505896,05:20:00,05:20:00,70032,20,NaN,0,0,0,welln,...,RTL42025-hmo45011-Weekday-01,Forest Hills,NaN,0.0,O903_-1,903_0026,1.0,NaN,Orange-3-0,0.0
3,70505896,05:22:00,05:22:00,70278,30,NaN,0,0,0,astao,...,RTL42025-hmo45011-Weekday-01,Forest Hills,NaN,0.0,O903_-1,903_0026,1.0,NaN,Orange-3-0,0.0
4,70505896,05:24:00,05:24:00,70030,40,NaN,0,0,0,sull,...,RTL42025-hmo45011-Weekday-01,Forest Hills,NaN,0.0,O903_-1,903_0026,1.0,NaN,Orange-3-0,0.0


In [25]:
print("Join 2/2: (precedente) + routes (on=route_id)")
st_trip_route = st_trip.merge(routes, on="route_id", how="left")
print("Shape:", st_trip_route.shape); display(st_trip_route.head())

Join 2/2: (precedente) + routes (on=route_id)
Shape: (2133080, 36)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,...,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order,route_fare_class,line_id,listed_route,network_id
0,70505896,05:15:00,05:15:00,70036,1,NaN,0,1,0,ogmnl,...,Rapid Transit,1.0,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020.0,Rapid Transit,line-Orange,NaN,rapid_transit
1,70505896,05:16:00,05:16:00,70034,10,NaN,0,0,0,mlmnl,...,Rapid Transit,1.0,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020.0,Rapid Transit,line-Orange,NaN,rapid_transit
2,70505896,05:20:00,05:20:00,70032,20,NaN,0,0,0,welln,...,Rapid Transit,1.0,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020.0,Rapid Transit,line-Orange,NaN,rapid_transit
3,70505896,05:22:00,05:22:00,70278,30,NaN,0,0,0,astao,...,Rapid Transit,1.0,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020.0,Rapid Transit,line-Orange,NaN,rapid_transit
4,70505896,05:24:00,05:24:00,70030,40,NaN,0,0,0,sull,...,Rapid Transit,1.0,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020.0,Rapid Transit,line-Orange,NaN,rapid_transit


### 2.7 Aggregazione semplice: **eventi per route**

In [26]:
by_route = (st_trip_route.groupby("route_id")
                        .agg(n_stop_events=("stop_id","count"),
                             n_trips=("trip_id","nunique"))
                        .reset_index()
                        .sort_values("n_stop_events", ascending=False))
print("Prime righe:")
display(by_route.head())

Prime righe:


,route_id,n_stop_events,n_trips
93,Green-E,68522,3256
90,Green-B,68333,3010
91,Green-C,60757,3084
92,Green-D,60113,3091
95,Orange,37706,2196


### 2.8 Quante **route per stop** (`n_routes`)

In [27]:
route_per_stop = (st_trip_route.groupby("stop_id")["route_id"]
                                .nunique().reset_index(name="n_routes"))
stops_enriched = stops.merge(route_per_stop, on="stop_id", how="left").fillna({"n_routes":0})
print("stops_enriched HEAD:")
display(stops_enriched.head())

stops_enriched HEAD:


,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type,n_routes
0,1,1.0,Washington St opp Ruggles St,NaN,NaN,NaN,42.330957,-71.082754,ExpressBus-Downtown,NaN,https://www.mbta.com/stops/1,NaN,0,NaN,1,Boston,Washington Street,Ruggles Street,3.0,0.0
1,10,10.0,Theo Glynn Way @ Newmarket Sq,NaN,NaN,NaN,42.330555,-71.068787,LocalBus,NaN,https://www.mbta.com/stops/10,NaN,0,NaN,1,Boston,Theodore Glynn Way,Newmarket Square,3.0,0.0
2,10000,10000.0,Tremont St opp Temple Pl,NaN,NaN,NaN,42.355692,-71.062911,LocalBus,NaN,https://www.mbta.com/stops/10000,NaN,0,NaN,1,Boston,Tremont Street,Temple Place,3.0,5.0
3,10003,10003.0,Albany St opp Randall St,NaN,NaN,NaN,42.331591,-71.076237,LocalBus,NaN,https://www.mbta.com/stops/10003,NaN,0,NaN,1,Boston,Albany Street,Randall Street,3.0,0.0
4,10005,10005.0,Albany St opp E Concord St,NaN,NaN,NaN,42.335017,-71.071280,LocalBus,NaN,https://www.mbta.com/stops/10005,NaN,0,NaN,1,Boston,Albany Street,NaN,3.0,0.0


### 2.9 Orari HH:MM:SS → **minuti dal mezzanotte** (con esempi)

In [28]:
def hhmmss_to_minutes(x):
    if pd.isna(x): return np.nan
    parts = str(x).split(":")
    if len(parts) < 2: return np.nan
    h = int(parts[0]); m = int(parts[1]); s = int(parts[2]) if len(parts)>2 else 0
    return h*60 + m + s/60.0

print("Esempi:")
for ex in ["05:10:00","23:59:00","27:15:00"]:
    print(ex, "->", hhmmss_to_minutes(ex))

st_trip_route["arr_min"] = st_trip_route["arrival_time"].map(hhmmss_to_minutes)
print("Colonna arr_min creata. HEAD:")
display(st_trip_route[["arrival_time","arr_min"]].head())

Esempi:
05:10:00 -> 310.0
23:59:00 -> 1439.0
27:15:00 -> 1635.0
Colonna arr_min creata. HEAD:


,arrival_time,arr_min
0,05:15:00,315.0
1,05:16:00,316.0
2,05:20:00,320.0
3,05:22:00,322.0
4,05:24:00,324.0


### 2.10 Salvataggi **visibili** (percorsi + dimensioni)

In [29]:
import os

stops_clean_path = DATA_CLEAN / "gtfs_stops_clean.parquet"
routes_summary_path = DATA_CLEAN / "gtfs_routes_summary.parquet"

stops_enriched.to_parquet(stops_clean_path, index=False)
by_route.to_parquet(routes_summary_path, index=False)

print("Scritti:")
print(" -", stops_clean_path, f"({os.path.getsize(stops_clean_path)} bytes)")
print(" -", routes_summary_path, f"({os.path.getsize(routes_summary_path)} bytes)")

Scritti:
 - /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_01/clean/gtfs_stops_clean.parquet (548011 bytes)
 - /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_01/clean/gtfs_routes_summary.parquet (6786 bytes)


---
## 3) Esercizio Finale: Analisi Ore di Punta

Vogliamo rispondere a una domanda complessa: **Qual è l'ora di punta (con più passaggi/stop_events) per le 5 linee (route) più trafficate?**

Questo esercizio combina tutto:
1.  Caricamento dati (i file Parquet che abbiamo salvato).
2.  Join "implicito" (useremo `by_route` per filtrare `st_trip_route`).
3.  Manipolazione stringhe vettoriale (per estrarre l'ora da `arrival_time`).
4.  Aggregazione (`groupby` su due colonne).
5.  Rimodellamento (`unstack`) e analisi (`idxmax`).

In [30]:
def find_peak_hours_for_routes(all_data_df, route_ids_list, verbose=True):
    df = all_data_df[all_data_df["route_id"].isin(route_ids_list)].copy()
    if df.empty:
        if verbose: print(f"Nessun dato per: {route_ids_list}")
        return pd.DataFrame(index=route_ids_list, columns=["peak_hour","peak_count"], dtype="Int64")

    # usa arrival_time con fallback a departure_time
    time_col = df["arrival_time"].fillna(df.get("departure_time"))
    # estrai le ore in modo veloce e tollerante
    hour = pd.to_numeric(time_col.str.slice(0, 2), errors="coerce")
    df["hour_of_day"] = (hour % 24).astype("Int64")

    # scarta righe senza ora e logga quante
    dropped = df["hour_of_day"].isna().sum()
    df = df.dropna(subset=["hour_of_day"])
    df["hour_of_day"] = df["hour_of_day"].astype(int)

    # tabella route x ora con tutte le 24 colonne garantite
    hourly_counts = (
        df.groupby(["route_id", "hour_of_day"]).size()
          .unstack("hour_of_day", fill_value=0)
          .reindex(columns=range(24), fill_value=0)
    )

    peak_hour = hourly_counts.idxmax(axis=1).astype("Int64").rename("peak_hour")
    peak_count = hourly_counts.max(axis=1).astype("Int64").rename("peak_count")

    out = pd.concat([peak_hour, peak_count], axis=1)
    # assicura presenza di tutte le route richieste 
    out = out.reindex(route_ids_list)

    if verbose:
        print(f"Righe considerate: {len(df)} | scartate per ora non valida: {dropped}")
    return out


In [31]:
print("\nCalcolo delle 5 route più trafficate in corso...")
by_route = (st_trip_route.groupby("route_id")
                        .agg(n_stop_events=("stop_id","count"))
                        .reset_index()
                        .sort_values("n_stop_events", ascending=False))

# Estraiamo la lista di ID
top_5_routes_list = by_route.nlargest(5, "n_stop_events")["route_id"].to_list()
print(f"Top 5 trovate: {top_5_routes_list}")

risultati = find_peak_hours_for_routes(st_trip_route, top_5_routes_list)
print("\nRISULTATI PER 'TOP 5':")
display(risultati)


Calcolo delle 5 route più trafficate in corso...
Top 5 trovate: ['Green-E', 'Green-B', 'Green-C', 'Green-D', 'Orange']
Righe considerate: 295431 | scartate per ora non valida: 0

RISULTATI PER 'TOP 5':


,peak_hour,peak_count
route_id,,
Green-E,14,3721
Green-B,9,3676
Green-C,12,3343
Green-D,12,3337
Orange,7,2121
